# WoW Analytics - Data Explorer

This notebook explores the auction data stored as Parquet files using DuckDB.

DuckDB provides excellent support for:
- Reading Parquet files directly (including glob patterns)
- SQL-based analytics
- Efficient columnar operations

In [ ]:
################
# Setup DuckDB #
################

import duckdb
from duckdb import SQLExpression

# Create in-memory DuckDB connection
duck = duckdb.connect()

# Configure your data directory here
# For local storage:
DATA_DIR = "/Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data/eu"

# For S3 storage (DuckDB can read from S3 directly):
# DATA_DIR = "s3://your-bucket/wow-analytics"

print(f"Data directory: {DATA_DIR}")

In [ ]:
##############################
# Check connected realms     #
##############################

# Read connected realms metadata
realms_path = f"{DATA_DIR}/global/connected_realms.parquet"

connected_realms = duck.read_parquet(realms_path)

# Show all realms
print("All connected realms:")
connected_realms.show(max_rows=10)

In [ ]:
##############################
# Find a specific realm      #
##############################

# Search for a realm by name/slug
search_term = "elune"  # Change this to search for different realms

realm = connected_realms.filter(
    SQLExpression(f"realm_slugs LIKE '%{search_term}%'")
)
realm.show()

# Get the realm ID for further queries
realm_df = realm.fetchdf()
if len(realm_df) > 0:
    REALM_ID = realm_df.iloc[0]["id"]
    print(f"\nRealm ID: {REALM_ID}")

In [ ]:
##############################
# Query auctions for a realm #
##############################

# Use glob pattern to read all auction files for a realm
# Pattern: auctions/{date}/{hour}/realm_{id}/*.parquet

realm_id = REALM_ID if 'REALM_ID' in dir() else 1315  # Default to Elune/Varimathras

auctions_path = f"{DATA_DIR}/auctions/*/*/realm_{realm_id}/*.parquet"

print(f"Reading auctions from: {auctions_path}")
auctions = duck.read_parquet(auctions_path)
auctions.show(max_rows=10)

In [ ]:
##############################
# Auction statistics         #
##############################

# Count auctions by date
query = f"""
SELECT 
    date,
    COUNT(*) as num_auctions,
    COUNT(DISTINCT item_id) as unique_items,
    SUM(quantity) as total_quantity,
    AVG(unit_price) / 10000 as avg_price_gold
FROM read_parquet('{auctions_path}')
GROUP BY date
ORDER BY date DESC
"""

print("Auction statistics by date:")
duck.sql(query).show()

In [ ]:
##############################
# Top items by volume        #
##############################

query = f"""
SELECT 
    item_id,
    COUNT(*) as num_listings,
    SUM(quantity) as total_quantity,
    MIN(unit_price) / 10000 as min_price_gold,
    AVG(unit_price) / 10000 as avg_price_gold,
    MAX(unit_price) / 10000 as max_price_gold
FROM read_parquet('{auctions_path}')
WHERE unit_price IS NOT NULL
GROUP BY item_id
ORDER BY total_quantity DESC
LIMIT 20
"""

print("Top 20 items by total quantity:")
duck.sql(query).show()

In [ ]:
##############################
# Query commodities          #
##############################

commodities_path = f"{DATA_DIR}/auctions/*/*/commodities/*.parquet"

query = f"""
SELECT 
    date,
    hour,
    COUNT(*) as num_listings,
    COUNT(DISTINCT item_id) as unique_items
FROM read_parquet('{commodities_path}')
GROUP BY date, hour
ORDER BY date DESC, hour DESC
LIMIT 10
"""

print("Commodity auction snapshots:")
duck.sql(query).show()

In [ ]:
##############################
# Price history for an item  #
##############################

# Example: Track price history for a specific item
item_id = 194755  # Change to the item you want to track

query = f"""
SELECT 
    date,
    hour,
    COUNT(*) as num_listings,
    SUM(quantity) as total_quantity,
    MIN(unit_price) / 10000 as min_price_gold,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_price) / 10000 as median_price_gold,
    MAX(unit_price) / 10000 as max_price_gold
FROM read_parquet('{auctions_path}')
WHERE item_id = {item_id}
  AND unit_price IS NOT NULL
GROUP BY date, hour
ORDER BY date, hour
"""

print(f"Price history for item {item_id}:")
duck.sql(query).show()

In [ ]:
##############################
# Cross-realm comparison     #
##############################

# Compare an item's price across multiple realms
all_auctions_path = f"{DATA_DIR}/auctions/*/*/realm_*/*.parquet"
item_id = 194755  # Change to the item you want to compare

query = f"""
SELECT 
    connected_realm_id,
    COUNT(*) as num_listings,
    SUM(quantity) as total_quantity,
    MIN(unit_price) / 10000 as min_price_gold,
    AVG(unit_price) / 10000 as avg_price_gold
FROM read_parquet('{all_auctions_path}')
WHERE item_id = {item_id}
  AND unit_price IS NOT NULL
GROUP BY connected_realm_id
ORDER BY min_price_gold ASC
LIMIT 10
"""

print(f"Price comparison for item {item_id} across realms:")
duck.sql(query).show()